In [ ]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
from pathlib import Path
import numpy as np
import mmcv
import matplotlib.pyplot as plt
import cv2

# CRATERI:
# config_file = 'checkpoints/Craters/cascade_mask50/cascade_mask50_2022-08-05T15:15:10_768_25e_Craters_lr_0.002_CosineAnnealing/Cascade_Mask_RCNN.py'
# checkpoint_file = 'checkpoints/Craters/cascade_mask50/cascade_mask50_2022-08-05T15:15:10_768_25e_Craters_lr_0.002_CosineAnnealing/epoch_10.pth'

# RETINASWIN MULTIWAKE:
config_file = 'checkpoints/B4/retina_swin/retina_swin_2022-07-28T22:15:36_768_50e_B4_lr_0.001_step/retinanet_swin-t-p4-w7_fpn_1x_coco.py'
checkpoint_file = 'checkpoints/B4/retina_swin/retina_swin_2022-07-28T22:15:36_768_50e_B4_lr_0.001_step/epoch_44.pth'


# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

In [1]:
import os
import fiftyone as fo

IMAGES_DIR='/home/roberto/PythonProjects/S2RAWVessel/mmdetection/data/MS3/imgs'
JSON_PATH='/home/roberto/PythonProjects/S2RAWVessel/mmdetection/data/MS3/annotations/test.json'
LABEL='vessel'
# Load COCO formatted dataset
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=IMAGES_DIR,
    labels_path=JSON_PATH,
    include_id=True,
    label_field=LABEL,)
# Verify that the class list for our dataset was imported
# print(coco_dataset.default_classes)  # ['airplane', 'apple', ...]
print(dataset)

 100% |███████████████| 1390/1390 [17.0s elapsed, 0s remaining, 77.1 samples/s]      
Name:        2023.09.01.10.57.48
Media type:  image
Num samples: 1390
Persistent:  False
Tags:        []
Sample fields:
    id:                   fiftyone.core.fields.ObjectIdField
    filepath:             fiftyone.core.fields.StringField
    tags:                 fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:             fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    vessel_detections:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    vessel_segmentations: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    vessel_coco_id:       fiftyone.core.fields.IntField


In [ ]:
# img_path = Path('data/DATASETS/Craters/val2017/539.jpg')
# img = cv2.imread(img_path.as_posix())
# shape = mmcv.imread(img_path).shape

# result = inference_detector(model, img)
# img = cv2.imread('/home/sirbastiano/Scaricati/tree-736885__480.jpg')

# type(img)
# img.shape
# boxes, masks = result
# plt.imshow(masks[0][5])
# len(boxes[0])
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches

# fig,ax = plt.subplots(figsize=(10,10))
# ax.imshow(img)
# for item in boxes[0]:
#      X=item
#      topleft = [X[0],X[1]]
#      botright = [X[2],X[3]]
#      score=X[4]

#      rect = patches.Rectangle((topleft[0], topleft[1]), botright[0]-topleft[0], botright[1]-topleft[1], linewidth=3, edgecolor='r', facecolor='none')
#      ax.add_patch(rect)
# plt.show()
# import fiftyone as fo
segmentation = False

for sample in dataset:
    # sample = fo.Sample(filepath="data/DATASETS/Craters/val2017/539.jpg")
    # img = cv2.imread("data/DATASETS/Craters/val2017/539.jpg")
    img = cv2.imread(sample.filepath)
    h,w,c=img.shape
    result = inference_detector(model, img)

    if segmentation:
        boxes, masks = result # if instance segmentation method
    else:
        boxes = result

    X = boxes[0]
    sample["RetinaSwin"] = fo.Detections(
        detections=[
            fo.Detection(label=LABEL, confidence=X[4], bounding_box=[X[0]/w, X[1]/h, X[2]/w-X[0]/w, X[3]/h-X[1]/h]) for X in boxes[0]
        ]
    )
    sample.save()
    
    # print(sample)
    # dataset.add_sample(sample)

In [2]:
session = fo.launch_app(dataset)


Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v0.21.6

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|



In [ ]:
from fiftyone import ViewField as F

# Only contains detections with confidence >= 0.75
high_conf_view = dataset.filter_labels("RetinaSwin", F("confidence") > 0.65)

In [ ]:
# Evaluate the predictions in the `faster_rcnn` field of our `high_conf_view`
# with respect to the objects in the `ground_truth` field
results = high_conf_view.evaluate_detections(
    "RetinaSwin",
    gt_field=f"{LABEL}_detections",
    eval_key="eval",
    compute_mAP=True,
)

In [ ]:
# Get the 10 most common classes in the dataset
counts = dataset.count_values(f"{LABEL}_detections.detections.label")
classes_top10 = sorted(counts, key=counts.get, reverse=True)[:10]

# Print a classification report for the top-10 classes
results.print_report(classes=classes_top10)
print('Mean Average Precision:' results.mAP())

In [ ]:
plot = results.plot_pr_curves(classes=[LABEL])
plot.show()

In [ ]:
# from mmdet.apis import init_detector, inference_detector, show_result_pyplot
# from pathlib import Path
# import numpy as np
# import mmcv
# import matplotlib.pyplot as plt
# import cv2
# import os
# import fiftyone as fo
# from fiftyone import ViewField as F
# global LABEL, MODEL_NAME

# def model_load(model_cp, config):
#      model = init_detector(config, model_cp, device='cuda:0')
#      return model

# def dataset_parser(IMAGES_DIR,JSON_PATH,LABEL):
#      # Load COCO formatted dataset
#      dataset = fo.Dataset.from_dir(
#      dataset_type=fo.types.COCODetectionDataset,
#      data_path=IMAGES_DIR,
#      labels_path=JSON_PATH,
#      include_id=True,
#      label_field='GT',)
#      # Verify that the class list for our dataset was imported
#      # print(coco_dataset.default_classes)  # ['airplane', 'apple', ...]
#      print(dataset)
#      return dataset

# # MODEL
# CFG = 'checkpoints/Craters/cascade_mask50/cascade_mask50_2022-08-05T15:15:10_768_25e_Craters_lr_0.002_CosineAnnealing/Cascade_Mask_RCNN.py'
# CHP = 'checkpoints/Craters/cascade_mask50/cascade_mask50_2022-08-05T15:15:10_768_25e_Craters_lr_0.002_CosineAnnealing/epoch_10.pth'
# MODEL_NAME = 'Cascade_Mask_RCNN'
# model = model_load(model_cp=CHP, config=CFG)
# # DATASET
# IMAGES_DIR='data/DATASETS/Craters/val2017'
# JSON_PATH='data/DATASETS/Craters/annotations/instances_val2017.json'
# LABEL='crater'
# dataset = dataset_parser(IMAGES_DIR,JSON_PATH,LABEL)

# for sample in dataset:
#      # sample = fo.Sample(filepath="data/DATASETS/Craters/val2017/539.jpg")
#     # img = cv2.imread("data/DATASETS/Craters/val2017/539.jpg")
#     img = cv2.imread(sample.filepath)
#     h,w,c=img.shape
#     result = inference_detector(model, img)
#     boxes, masks = result
#     X = boxes[0]

#     sample[MODEL_NAME] = fo.Detections(
#         detections=[
#             fo.Detection(label=LABEL, confidence=X[4], bounding_box=[X[0]/w, X[1]/h, X[2]/w-X[0]/w, X[3]/h-X[1]/h]) for X in boxes[0]
#         ]
#     )
#     sample.save()

In [ ]:
session = fo.launch_app(dataset)

In [ ]:
# Only contains detections with confidence >= 0.75
high_conf_view = dataset.filter_labels(MODEL_NAME, F("confidence") > 0.75)

# Evaluate the predictions in the `faster_rcnn` field of our `high_conf_view`
# with respect to the objects in the `ground_truth` field
results = high_conf_view.evaluate_detections(
    MODEL_NAME,
    gt_field="GT_detections",
    eval_key="eval",
    compute_mAP=True,
    iou=0.5,
)

# Get the 10 most common classes in the dataset
counts = dataset.count_values("GT_detections.detections.label")
classes_top10 = sorted(counts, key=counts.get, reverse=True)[:10]

# Print a classification report for the top-10 classes
# with open(f"{MODEL_NAME}_PR.txt", 'w') as f: 
#      results.print_report(classes=classes_top10)
#      f.write(results.print_report(classes=classes_top10))


print(results.mAP())

plot = results.plot_pr_curves(classes=[LABEL])
plot.show()

In [ ]:

# Print a classification report for the top-10 classes
with open(f"{MODEL_NAME}_PR.txt", 'w') as f: 
     # results.print_report(classes=classes_top10)
     f.write(results.to_str())

In [ ]:
results.serialize()

In [ ]:
results.report()

In [ ]:
fig=results.plot_confusion_matrix(backend='matplotlib',)
fig.savefig('Images/IAC22/confusion_matrix.png', dpi=300)